# Extract Foercasted Weather Data
# -------------------------------

## Initial Imports

In [1]:
# Import Dependencies
import requests
import pandas as pd
import json
import numpy as np
import datetime
from dateutil import tz

In [2]:
# Import API Key
from config import weather_api_key

In [3]:
# Documentation
# https://openweathermap.org/api/one-call-api

# Define the Functions
# --------------------

In [4]:
def convert_DateTime_UTC_to_CST(UTC_datetime_list, list_range):
    CST_datetime_list = []

    for date in list_range:    
        # Convert the date/time to ISO standard in string format
        date_time = datetime.datetime.utcfromtimestamp(UTC_datetime_list[date]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Create a datetime object, representing the UTC time
        time_utc = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')

        # Replace the timezone field of the datetime object to UTC
        from_zone = tz.gettz('UTC')
        time_utc = time_utc.replace(tzinfo=from_zone)

        # Convert time zone from UTC to central
        to_zone = tz.gettz('America/Chicago')
        time_cst = time_utc.astimezone(to_zone)

        # Append the date/time, year, month, day, and hour
        CST_datetime_list.append({
            "UTC_Time": UTC_datetime_list[date],
            "Date_Time": time_cst.strftime('%Y-%m-%d %H:%M:%S'),
            "Year": time_cst.year,
            "Month":time_cst.month,
            "Day":time_cst.day,
            "Hour":time_cst.hour
            })

    datetimeDataFrame = pd.DataFrame(CST_datetime_list)
    
    return datetimeDataFrame

In [5]:
def calculate_sunhour(sunrise_list, sunset_list, list_range):
    
    sunhour_list = []

    for day in list_range:
        # Convert the date/time to ISO standard in string format
        sunrise_date_time = datetime.datetime.utcfromtimestamp(sunrise_list[day]).strftime('%Y-%m-%d %H:%M:%S')
        sunset_date_time = datetime.datetime.utcfromtimestamp(sunset_list[day]).strftime('%Y-%m-%d %H:%M:%S')

        # Create a datetime object, representing the UTC time
        sunrise_utc = datetime.datetime.strptime(sunrise_date_time, '%Y-%m-%d %H:%M:%S')
        sunset_utc = datetime.datetime.strptime(sunset_date_time, '%Y-%m-%d %H:%M:%S')

        # Replace the timezone field of the datetime object to UTC
        from_zone = tz.gettz('UTC')
        
        sunrise_utc = sunrise_utc.replace(tzinfo=from_zone)
        sunset_utc = sunset_utc.replace(tzinfo=from_zone)

        # Convert time zone from UTC to central
        to_zone = tz.gettz('America/Chicago')
        
        sunrise_cst = sunrise_utc.astimezone(to_zone)
        sunset_cst = sunset_utc.astimezone(to_zone)
        
        # Convert to string
        sunrise_str = sunrise_cst.strftime('%Y-%m-%d %H:%M:%S')
        sunset_str = sunset_cst.strftime('%Y-%m-%d %H:%M:%S')

        # Calculate Sunhour
        sunrise = datetime.datetime.strptime(sunrise_str, '%Y-%m-%d %H:%M:%S')
        sunset = datetime.datetime.strptime(sunset_str, '%Y-%m-%d %H:%M:%S')
        Sunhour_timedelta = sunset - sunrise
        Sunhour_seconds = Sunhour_timedelta.seconds
        Sunhour = Sunhour_seconds / 3600

        # Append to List
        sunhour_list.append({
            "Sunrise": sunrise_list[day],
            "Sunhour": Sunhour
        })
        
    sunhourDataFrame = pd.DataFrame(sunhour_list)
    
    return sunhourDataFrame

# Webberville Solar Farm
# ----------------------

In [6]:
# Geographical coordinates (latitude, longitude)
lat = "30.238333"
lon = "-97.508611"

# Exclude some parts of the weather data from the API response
    # It should be a comma-delimited list (without spaces)
    # Options: current,minutely,hourly,daily,alerts
part = "minutely,alerts"

# Define the Units of measurement
    # Options: stadnard,metric,imperial
units = "imperial"

# Make a request to openweathermap
requestURL = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&units={units}&appid={weather_api_key}"

# print(requestURL)

In [7]:
response = requests.get(requestURL)

if response.status_code == 200:
    # Turn the response into a JSON object
    responseJson = response.json()
    print("Successfully turned response into JSON object.")
else:
    # Else, print the Error status code
    print("The Error Status Code is:") 
    print(response.status_code)

Successfully turned response into JSON object.


# Current Weather Data:

## Date / Time

In [8]:
# Convert the date/time to ISO standard in string format
current_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["dt"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
current_time_utc = datetime.datetime.strptime(current_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
current_time_utc = current_time_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
current_time_cst = current_time_utc.astimezone(to_zone)

# Display the current time
current_time_cst

datetime.datetime(2020, 11, 7, 3, 7, 38, tzinfo=tzfile('US/Central'))

In [9]:
# Extract the date & time and store as a string
current_Date_Time = current_time_cst.strftime('%Y-%m-%d %H:%M:%S')

# Display the date & time as string
current_Date_Time

'2020-11-07 03:07:38'

In [10]:
# Extract the date and store as a string
current_Date = datetime.datetime.date(current_time_cst)
current_Date = current_Date.strftime('%Y-%m-%d')

# Display the date in %Y-%m-%d
current_Date

'2020-11-07'

In [11]:
# Exract the time in %H:%M:%S
current_Time = current_time_cst.strftime('%H:%M:%S')

# Display the hour
current_Time

'03:07:38'

In [12]:
# Extract the year from the current time
current_Year = current_time_cst.year

# Display the year
current_Year

2020

In [13]:
# Extract the month from the current time
current_Month = current_time_cst.month

# Display the year
current_Month

11

In [14]:
# Extract the day from the current time
current_Day = current_time_cst.day

# Display the day
current_Day

7

In [15]:
# Extract the hour from the current time
current_Hour = current_time_cst.hour

# Display the day
current_Hour

3

In [16]:
# Alternative:
# Extract the hour from the current time
current_Hour = datetime.datetime.time(current_time_cst).hour

# Display the hour
current_Hour

3

In [17]:
# Store the next day as a variable
next_day = datetime.date(current_Year, current_Month, current_Day) + datetime.timedelta(days=1)
next_day

datetime.date(2020, 11, 8)

## Weather Description

In [18]:
# Extract the current weather description
current_Weather_Description = responseJson["current"]["weather"][0]["description"]

# Display the weather description
current_Weather_Description

'few clouds'

## Temperature

In [19]:
# Extract the current temperature (F)
current_Temperature_F = responseJson["current"]["temp"]

# Display the temperature
current_Temperature_F

53.13

## Sunhour

In [20]:
# Sunrise:

# Convert the date/time to ISO standard in string format
sunrise_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["sunrise"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
sunrise_utc = datetime.datetime.strptime(sunrise_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
sunrise_utc = sunrise_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
todays_sunrise = sunrise_utc.astimezone(to_zone)

# Convert to string
todays_sunrise = todays_sunrise.strftime('%Y-%m-%d %H:%M:%S')

# Display sunrise time
todays_sunrise

'2020-11-07 06:49:28'

In [21]:
# Sunset:

# Convert the date/time to ISO standard in string format
sunset_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["sunset"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
sunset_utc = datetime.datetime.strptime(sunset_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
sunset_utc = sunset_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
todays_sunset = sunset_utc.astimezone(to_zone)

# Convert to string
todays_sunset = todays_sunset.strftime('%Y-%m-%d %H:%M:%S')

# Display sunrise time
todays_sunset

'2020-11-07 17:38:06'

In [22]:
todays_sunrise = datetime.datetime.strptime(todays_sunrise, '%Y-%m-%d %H:%M:%S')
todays_sunset = datetime.datetime.strptime(todays_sunset, '%Y-%m-%d %H:%M:%S')
Sunhour_timedelta = todays_sunset - todays_sunrise
print(f"The sunhour time in %H:%M:%S : {Sunhour_timedelta}")
Sunhour_seconds = Sunhour_timedelta.seconds
todays_Sunhour = Sunhour_seconds / 3600
print(f"The sunhour time in hrs : {todays_Sunhour}")

The sunhour time in %H:%M:%S : 10:48:38
The sunhour time in hrs : 10.810555555555556


## Cloud Cover

In [23]:
# Extract the current cloud cover (%)
current_CloudCover_percent = responseJson["current"]["clouds"]

# Display the Cloudiness, %
current_CloudCover_percent

20

## uv Index

In [24]:
# Extract the current UV Index
todays_uvIndex = responseJson["current"]["uvi"]

# Display the Midday UV index
todays_uvIndex

5.25

## Humidity

In [25]:
# Extract the current humidity (%)
current_Humidity_percent = responseJson["current"]["humidity"]

# Display the Cloudiness, %
current_Humidity_percent

93

## Current Weather DataFrame

In [26]:
current_weather_DF = pd.DataFrame([{
    "Date_Time": current_Date_Time,
    "Year": current_Year, 
    "Month": current_Month, 
    "Day": current_Day, 
    "Hour": current_Hour, 
    "Temperature_F": current_Temperature_F, 
    "Humidity_percent": current_Humidity_percent,
    "Sunhour": todays_Sunhour, 
    "CloudCover_percent": current_CloudCover_percent,
    "uvIndex": todays_uvIndex,
    "Weather_Description": current_Weather_Description
}])

current_weather_DF["Date_Time"] = pd.to_datetime(current_weather_DF["Date_Time"])

current_weather_DF

,Date_Time,Year,Month,Day,Hour,Temperature_F,Humidity_percent,Sunhour,CloudCover_percent,uvIndex,Weather_Description
0,2020-11-07 03:07:38,2020,11,7,3,53.13,93,10.810556,20,5.25,few clouds


In [27]:
current_weather_DF.dtypes

Date_Time              datetime64[ns]
Year                            int64
Month                           int64
Day                             int64
Hour                            int64
Temperature_F                 float64
Humidity_percent                int64
Sunhour                       float64
CloudCover_percent              int64
uvIndex                       float64
Weather_Description            object
dtype: object

# Daily Forecasted Weather Data:

## Create a dataframe for the daily forecasted weather data

In [28]:
forecasted_daily_weather = []

for day in np.arange(0, 8, 1):
    forecasted_daily_weather.append({
        "UTC_Time": responseJson["daily"][day]["dt"],
        "Sunrise": responseJson["daily"][day]["sunrise"],
        "Sunset": responseJson["daily"][day]["sunset"],
        "uvIndex": responseJson["daily"][day]["uvi"]
    })

daily_weather_DF = pd.DataFrame(forecasted_daily_weather)
daily_weather_DF

,UTC_Time,Sunrise,Sunset,uvIndex
0,1604772000,1604753368,1604792286,5.25
1,1604858400,1604839817,1604878647,5.39
2,1604944800,1604926267,1604965009,5.33
3,1605031200,1605012717,1605051373,5.13
4,1605117600,1605099167,1605137738,5.25
5,1605204000,1605185618,1605224105,5.20
6,1605290400,1605272068,1605310473,5.07
7,1605376800,1605358519,1605396843,4.30


## Clean up the UTC_Time column: Convert UTC time to CST

In [29]:
daily_index = np.arange(0,8,1)

daily_UTC_date_time = daily_weather_DF["UTC_Time"]

date_time_DF = convert_DateTime_UTC_to_CST(daily_UTC_date_time, daily_index)

date_time_DF

,UTC_Time,Date_Time,Year,Month,Day,Hour
0,1604772000,2020-11-07 12:00:00,2020,11,7,12
1,1604858400,2020-11-08 12:00:00,2020,11,8,12
2,1604944800,2020-11-09 12:00:00,2020,11,9,12
3,1605031200,2020-11-10 12:00:00,2020,11,10,12
4,1605117600,2020-11-11 12:00:00,2020,11,11,12
5,1605204000,2020-11-12 12:00:00,2020,11,12,12
6,1605290400,2020-11-13 12:00:00,2020,11,13,12
7,1605376800,2020-11-14 12:00:00,2020,11,14,12


In [30]:
daily_weather_DF = pd.merge(daily_weather_DF, date_time_DF, on='UTC_Time', how='outer')
daily_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
daily_weather_DF["Date_Time"] = pd.to_datetime(daily_weather_DF["Date_Time"])
daily_weather_DF

,Sunrise,Sunset,uvIndex,Date_Time,Year,Month,Day,Hour
0,1604753368,1604792286,5.25,2020-11-07 12:00:00,2020,11,7,12
1,1604839817,1604878647,5.39,2020-11-08 12:00:00,2020,11,8,12
2,1604926267,1604965009,5.33,2020-11-09 12:00:00,2020,11,9,12
3,1605012717,1605051373,5.13,2020-11-10 12:00:00,2020,11,10,12
4,1605099167,1605137738,5.25,2020-11-11 12:00:00,2020,11,11,12
5,1605185618,1605224105,5.20,2020-11-12 12:00:00,2020,11,12,12
6,1605272068,1605310473,5.07,2020-11-13 12:00:00,2020,11,13,12
7,1605358519,1605396843,4.30,2020-11-14 12:00:00,2020,11,14,12


## Use sunrise and sunset times to calculate the total hours of sunhour for each day

In [31]:
daily_index = np.arange(0,8,1)

UTC_sunrise = daily_weather_DF["Sunrise"]
UTC_sunset = daily_weather_DF["Sunset"]

sunhour_DF = calculate_sunhour(UTC_sunrise, UTC_sunset, daily_index)

sunhour_DF

,Sunrise,Sunhour
0,1604753368,10.810556
1,1604839817,10.786111
2,1604926267,10.761667
3,1605012717,10.737778
4,1605099167,10.714167
5,1605185618,10.690833
6,1605272068,10.668056
7,1605358519,10.645556


In [32]:
daily_weather_DF = pd.merge(daily_weather_DF, sunhour_DF, on='Sunrise', how='outer')
daily_weather_DF.drop(columns=["Sunrise", "Sunset"], axis=1, inplace=True)
daily_weather_DF

,uvIndex,Date_Time,Year,Month,Day,Hour,Sunhour
0,5.25,2020-11-07 12:00:00,2020,11,7,12,10.810556
1,5.39,2020-11-08 12:00:00,2020,11,8,12,10.786111
2,5.33,2020-11-09 12:00:00,2020,11,9,12,10.761667
3,5.13,2020-11-10 12:00:00,2020,11,10,12,10.737778
4,5.25,2020-11-11 12:00:00,2020,11,11,12,10.714167
5,5.20,2020-11-12 12:00:00,2020,11,12,12,10.690833
6,5.07,2020-11-13 12:00:00,2020,11,13,12,10.668056
7,4.30,2020-11-14 12:00:00,2020,11,14,12,10.645556


## Display the final daily forecasted weather dataframe

In [33]:
# Re-Order the columns
daily_weather_DF = daily_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Sunhour","uvIndex"]]
daily_weather_DF

,Date_Time,Year,Month,Day,Hour,Sunhour,uvIndex
0,2020-11-07 12:00:00,2020,11,7,12,10.810556,5.25
1,2020-11-08 12:00:00,2020,11,8,12,10.786111,5.39
2,2020-11-09 12:00:00,2020,11,9,12,10.761667,5.33
3,2020-11-10 12:00:00,2020,11,10,12,10.737778,5.13
4,2020-11-11 12:00:00,2020,11,11,12,10.714167,5.25
5,2020-11-12 12:00:00,2020,11,12,12,10.690833,5.20
6,2020-11-13 12:00:00,2020,11,13,12,10.668056,5.07
7,2020-11-14 12:00:00,2020,11,14,12,10.645556,4.30


In [34]:
# Check the data types
daily_weather_DF.dtypes

Date_Time    datetime64[ns]
Year                  int64
Month                 int64
Day                   int64
Hour                  int64
Sunhour             float64
uvIndex             float64
dtype: object

## Clean the dataframe so it can be merged with the hourly forecasted data

In [35]:
daily_weather_DF.drop(columns=["Date_Time", "Year", "Month", "Hour"], axis=1, inplace=True)
daily_weather_DF

C:\Users\rahul\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Day,Sunhour,uvIndex
0,7,10.810556,5.25
1,8,10.786111,5.39
2,9,10.761667,5.33
3,10,10.737778,5.13
4,11,10.714167,5.25
5,12,10.690833,5.20
6,13,10.668056,5.07
7,14,10.645556,4.30


# Hourly Forecasted Weather Data:

In [36]:
forecasted_hourly_weather = []

for hour in np.arange(0, 48, 1):
    forecasted_hourly_weather.append({
        "UTC_Time": responseJson["hourly"][hour]["dt"],
        "Temperature_F": responseJson["hourly"][hour]["temp"],
        "Weather_Description": responseJson["hourly"][hour]["weather"][0]["description"],
        "CloudCover_percent": responseJson["hourly"][hour]["clouds"],
        "Humidity_percent": responseJson["hourly"][hour]["humidity"]
    })

hourly_weather_DF = pd.DataFrame(forecasted_hourly_weather)
hourly_weather_DF.head()

,UTC_Time,Temperature_F,Weather_Description,CloudCover_percent,Humidity_percent
0,1604739600,53.13,few clouds,20,93
1,1604743200,55.60,clear sky,10,88
2,1604746800,56.80,clear sky,4,86
3,1604750400,57.29,clear sky,1,86
4,1604754000,57.54,clear sky,0,87


In [37]:
hourly_index = np.arange(0,48,1)

hourly_UTC_date_time = hourly_weather_DF["UTC_Time"]

hourly_date_time_DF = convert_DateTime_UTC_to_CST(hourly_UTC_date_time, hourly_index)

hourly_date_time_DF.head()

,UTC_Time,Date_Time,Year,Month,Day,Hour
0,1604739600,2020-11-07 03:00:00,2020,11,7,3
1,1604743200,2020-11-07 04:00:00,2020,11,7,4
2,1604746800,2020-11-07 05:00:00,2020,11,7,5
3,1604750400,2020-11-07 06:00:00,2020,11,7,6
4,1604754000,2020-11-07 07:00:00,2020,11,7,7


In [38]:
hourly_weather_DF = pd.merge(hourly_weather_DF, hourly_date_time_DF, on='UTC_Time', how='outer')
hourly_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
hourly_weather_DF["Date_Time"] = pd.to_datetime(hourly_weather_DF["Date_Time"])
hourly_weather_DF = hourly_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Temperature_F", "Humidity_percent", "CloudCover_percent", "Weather_Description"]]
hourly_weather_DF.head()

,Date_Time,Year,Month,Day,Hour,Temperature_F,Humidity_percent,CloudCover_percent,Weather_Description
0,2020-11-07 03:00:00,2020,11,7,3,53.13,93,20,few clouds
1,2020-11-07 04:00:00,2020,11,7,4,55.60,88,10,clear sky
2,2020-11-07 05:00:00,2020,11,7,5,56.80,86,4,clear sky
3,2020-11-07 06:00:00,2020,11,7,6,57.29,86,1,clear sky
4,2020-11-07 07:00:00,2020,11,7,7,57.54,87,0,clear sky


In [39]:
# Check the data types
hourly_weather_DF.dtypes

Date_Time              datetime64[ns]
Year                            int64
Month                           int64
Day                             int64
Hour                            int64
Temperature_F                 float64
Humidity_percent                int64
CloudCover_percent              int64
Weather_Description            object
dtype: object

# Merge the Daily and Hourly Forecasted Weather DataFrames:

In [40]:
forecasted_weather_DF = pd.merge(hourly_weather_DF, daily_weather_DF, on='Day', how='inner')
forecasted_weather_DF = forecasted_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Temperature_F", "Humidity_percent", "Sunhour", "CloudCover_percent", "uvIndex", "Weather_Description"]]

forecasted_weather_DF

,Date_Time,Year,Month,Day,Hour,Temperature_F,Humidity_percent,Sunhour,CloudCover_percent,uvIndex,Weather_Description
0,2020-11-07 03:00:00,2020,11,7,3,53.13,93,10.810556,20,5.25,few clouds
1,2020-11-07 04:00:00,2020,11,7,4,55.60,88,10.810556,10,5.25,clear sky
2,2020-11-07 05:00:00,2020,11,7,5,56.80,86,10.810556,4,5.25,clear sky
3,2020-11-07 06:00:00,2020,11,7,6,57.29,86,10.810556,1,5.25,clear sky
4,2020-11-07 07:00:00,2020,11,7,7,57.54,87,10.810556,0,5.25,clear sky
5,2020-11-07 08:00:00,2020,11,7,8,61.11,80,10.810556,0,5.25,clear sky
6,2020-11-07 09:00:00,2020,11,7,9,65.89,70,10.810556,0,5.25,clear sky
7,2020-11-07 10:00:00,2020,11,7,10,70.00,62,10.810556,0,5.25,clear sky
8,2020-11-07 11:00:00,2020,11,7,11,73.45,55,10.810556,0,5.25,clear sky
9,2020-11-07 12:00:00,2020,11,7,12,76.24,46,10.810556,0,5.25,clear sky


In [41]:
forecasted_weather_DF.dtypes

Date_Time              datetime64[ns]
Year                            int64
Month                           int64
Day                             int64
Hour                            int64
Temperature_F                 float64
Humidity_percent                int64
Sunhour                       float64
CloudCover_percent              int64
uvIndex                       float64
Weather_Description            object
dtype: object

# Hackberry Wind Farm
# -------------------

In [42]:
# Geographical coordinates (latitude, longitude)
lat = "32.776111"
lon = "-99.476444"

# Exclude some parts of the weather data from the API response
    # It should be a comma-delimited list (without spaces)
    # Options: current,minutely,hourly,daily,alerts
part = "minutely,alerts"

# Define the Units of measurement
    # Options: stadnard,metric,imperial
units = "imperial"

# Make a request to openweathermap
requestURL = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&units={units}&appid={weather_api_key}"

# print(requestURL)

In [43]:
response = requests.get(requestURL)

if response.status_code == 200:
    # Turn the response into a JSON object
    responseJson = response.json()
    print("Successfully turned response into JSON object.")
else:
    # Else, print the Error status code
    print("The Error Status Code is:") 
    print(response.status_code)

Successfully turned response into JSON object.


# Current Weather

In [44]:
current_date_time_UCT = [responseJson["current"]["dt"]]
current_weather_index = np.arange(0,len(current_date_time_UCT),1)

current_date_time_DF = convert_DateTime_UTC_to_CST(current_date_time_UCT, current_weather_index)

current_date_time_DF.head()

,UTC_Time,Date_Time,Year,Month,Day,Hour
0,1604740060,2020-11-07 03:07:40,2020,11,7,3


In [45]:
current_weather_DF = pd.DataFrame([{
    "UTC_Time": responseJson["current"]["dt"],
    "Temperature_F": responseJson["current"]["temp"], 
    "Weather_Description": responseJson["current"]["weather"][0]["description"],
    "Humidity_percent": responseJson["current"]["humidity"],
    "WindSpeed_mph": responseJson["current"]["wind_speed"],
    "WindDirection_degrees": responseJson["current"]["wind_deg"]
}])

current_weather_DF

,UTC_Time,Temperature_F,Weather_Description,Humidity_percent,WindSpeed_mph,WindDirection_degrees
0,1604740060,57.61,clear sky,87,11.41,150


In [46]:
current_weather_DF = pd.merge(current_weather_DF, current_date_time_DF, on='UTC_Time', how='inner')
current_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
current_weather_DF["Date_Time"] = pd.to_datetime(current_weather_DF["Date_Time"])
current_weather_DF = current_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Temperature_F", "Humidity_percent", "WindSpeed_mph", "WindDirection_degrees", "Weather_Description"]]
current_weather_DF

,Date_Time,Year,Month,Day,Hour,Temperature_F,Humidity_percent,WindSpeed_mph,WindDirection_degrees,Weather_Description
0,2020-11-07 03:07:40,2020,11,7,3,57.61,87,11.41,150,clear sky


# Forecasted Weather

In [47]:
forecasted_hourly_weather = []

for hour in np.arange(0, 48, 1):
    forecasted_hourly_weather.append({
        "UTC_Time": responseJson["hourly"][hour]["dt"],
        "Temperature_F": responseJson["hourly"][hour]["temp"],
        "Weather_Description": responseJson["hourly"][hour]["weather"][0]["description"],
        "Humidity_percent": responseJson["hourly"][hour]["humidity"],
        "WindSpeed_mph": responseJson["hourly"][hour]["wind_speed"],
        "WindDirection_degrees": responseJson["hourly"][hour]["wind_deg"]
    })

hourly_weather_DF = pd.DataFrame(forecasted_hourly_weather)
hourly_weather_DF.head()

,UTC_Time,Temperature_F,Weather_Description,Humidity_percent,WindSpeed_mph,WindDirection_degrees
0,1604739600,57.61,clear sky,87,11.65,166
1,1604743200,58.23,clear sky,89,11.54,165
2,1604746800,58.08,clear sky,88,10.51,172
3,1604750400,57.49,clear sky,88,9.66,172
4,1604754000,56.79,clear sky,89,8.79,161


In [48]:
hourly_index = np.arange(0,48,1)

hourly_UTC_date_time = hourly_weather_DF["UTC_Time"]

hourly_date_time_DF = convert_DateTime_UTC_to_CST(hourly_UTC_date_time, hourly_index)

hourly_date_time_DF.head()

,UTC_Time,Date_Time,Year,Month,Day,Hour
0,1604739600,2020-11-07 03:00:00,2020,11,7,3
1,1604743200,2020-11-07 04:00:00,2020,11,7,4
2,1604746800,2020-11-07 05:00:00,2020,11,7,5
3,1604750400,2020-11-07 06:00:00,2020,11,7,6
4,1604754000,2020-11-07 07:00:00,2020,11,7,7


In [49]:
hourly_weather_DF = pd.merge(hourly_weather_DF, hourly_date_time_DF, on='UTC_Time', how='outer')
hourly_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
hourly_weather_DF["Date_Time"] = pd.to_datetime(hourly_weather_DF["Date_Time"])
hourly_weather_DF = hourly_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Temperature_F", "Humidity_percent", "WindSpeed_mph", "WindDirection_degrees", "Weather_Description"]]
hourly_weather_DF

,Date_Time,Year,Month,Day,Hour,Temperature_F,Humidity_percent,WindSpeed_mph,WindDirection_degrees,Weather_Description
0,2020-11-07 03:00:00,2020,11,7,3,57.61,87,11.65,166,clear sky
1,2020-11-07 04:00:00,2020,11,7,4,58.23,89,11.54,165,clear sky
2,2020-11-07 05:00:00,2020,11,7,5,58.08,88,10.51,172,clear sky
3,2020-11-07 06:00:00,2020,11,7,6,57.49,88,9.66,172,clear sky
4,2020-11-07 07:00:00,2020,11,7,7,56.79,89,8.79,161,clear sky
5,2020-11-07 08:00:00,2020,11,7,8,58.87,83,9.89,161,clear sky
6,2020-11-07 09:00:00,2020,11,7,9,63.14,73,11.88,172,clear sky
7,2020-11-07 10:00:00,2020,11,7,10,67.17,65,12.24,179,clear sky
8,2020-11-07 11:00:00,2020,11,7,11,70.86,56,12.48,180,clear sky
9,2020-11-07 12:00:00,2020,11,7,12,73.09,50,12.37,169,clear sky


In [50]:
# Check the data types
hourly_weather_DF.dtypes

Date_Time                datetime64[ns]
Year                              int64
Month                             int64
Day                               int64
Hour                              int64
Temperature_F                   float64
Humidity_percent                  int64
WindSpeed_mph                   float64
WindDirection_degrees             int64
Weather_Description              object
dtype: object

# Weather Descriptions
# --------------------

In [51]:
weather_description_list = [
    # Clear
    'Clear',
    'Sunny',
    # Clouds
    'Partly cloudy', 
    'Cloudy', 
    'Overcast',
    # Drizzle
    'Freezing drizzle',
    'Light drizzle', 
    'Patchy light drizzle',
    # Snow
    'Patchy moderate snow', 
    'Moderate snow', 
    'Light sleet',
    'Light snow',  
    'Heavy snow', 
    'Patchy heavy snow', 
    'Blowing snow',  
    'Blizzard',
    'Ice pellets', 
    'Patchy light snow', 
    # Rain
    'Patchy rain possible',
    'Light rain shower', 
    'Moderate or heavy rain shower',
    'Moderate rain', 
    'Heavy rain', 
    'Light rain', 
    'Patchy light rain', 
    'Torrential rain shower',
    'Moderate rain at times', 
    'Heavy rain at times', 
    'Moderate or heavy freezing rain',
    # Thunderstorm
    'Thundery outbreaks possible',
    'Moderate or heavy rain with thunder',
    'Patchy light rain with thunder', 
    # Atmosphere
    'Mist',
    'Fog']

In [52]:
weather_condition_codes = [
    # Thunderstorm
    'thunderstorm with light rain',
    'thunderstorm with rain',
    'thunderstorm with heavy rain',
    'light thunderstorm',
    'thunderstorm',
    'heavy thunderstorm',
    'ragged thunderstorm',
    'thunderstorm with light drizzle',
    'thunderstorm with drizzle',
    'thunderstorm with heavy drizzle',
    # Drizzle
    'light intensity drizzle',
    'drizzle',
    'heavy intensity drizzle',
    'light intensity drizzle rain',
    'drizzle rain',
    'heavy intensity drizzle rain',
    'shower rain and drizzle',
    'heavy shower rain and drizzle',
    'shower drizzle',
    # Rain
    'light rain',
    'moderate rain',
    'heavy intensity rain',
    'very heavy rain',
    'extreme rain',
    'freezing rain',
    'light intensity shower rain',
    'shower rain',
    'heavy intensity shower rain',
    'ragged shower rain',
    # Snow
    'light snow',
    'Snow',
    'Heavy snow',
    'Sleet',
    'Light shower sleet',
    'Shower sleet',
    'Light rain and snow',
    'Rain and snow',
    'Light shower snow',
    'Shower snow',
    'Heavy shower snow',
    # Atmosphere
    'mist',
    'Smoke',
    'Haze',
    'sand/ dust whirls',
    'fog',
    'sand',
    'dust',
    'volcanic ash',
    'squalls',
    'tornado',
    # Clear
    'clear sky',
    # Clouds
    'few clouds',
    'scattered clouds',
    'broken clouds',
    'overcast clouds'   
]

In [54]:
weather_icons = [
    "clear sky",
    "few clouds",
    "scattered clouds",
    "broken clouds",
    "shower rain",
    "rain",
    "thunderstorm",
    "snow",
    "mist"
]